In [13]:
from urllib.request import urlretrieve
import os

import itk
import numpy as np

from itkwidgets import view
from ipywidgets import FloatProgress, Label, HBox

In [2]:
# Download data
file_name = '901-L.nrrd'
if not os.path.exists(file_name):
    url = 'https://data.kitware.com/api/v1/file/5ef373379014a6d84edb66f8/download'
    urlretrieve(url, file_name)

In [3]:
image = itk.imread(file_name)

In [36]:
camera = np.array([[-7.028664  , 20.177433  , 45.209637  ],
       [ 4.8549995 ,  9.24      , 48.065998  ],
       [-0.17604062,  0.06524051,  0.9822186 ]], dtype=np.float32)
camera = np.array([[-2.4597874 , 19.208582  , 45.884567  ],
       [ 4.855     ,  9.24      , 48.065998  ],
       [-0.22125234,  0.05076638,  0.97389436]], dtype=np.float32)
opacity_gaussians = [[{'position': 0.3,
   'height': 0.7545454545454545,
   'width': 0.2194444444444444,
   'xBias': 0.03777777777777777,
   'yBias': 0}]]
opacity_gaussians = [[{'position': 0.23888888888888893,
   'height': 0.3090909090909091,
   'width': 0.1722222222222221,
   'xBias': 0.04708953460902432,
   'yBias': 0}]]
viewer = view(image,
     cmap='bone_Matlab',
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     ui_collapsed=True,
     camera=camera,
     opacity_gaussians=opacity_gaussians
    )
viewer

Viewer(background=(0.6, 0.6, 0.6), camera=array([[-2.4597874 , 19.208582  , 45.884567  ],
       [ 4.855     ,…

In [16]:
label = Label('Segment bones')
progress = FloatProgress(value=0.0,
                         min=0.0,
                         max=1.0,
                         step=0.001,
                         description='progress:',
                         bar_style='info')
progress
box = HBox([label, progress])
box

In [10]:
bone_segmenter = itk.SegmentBonesInMicroCTFilter.New(image)
cortical_thickness=0.1
bone_segmenter.SetCorticalBoneThickness(cortical_thickness)

def update_progress():
    progress.value = bone_segmenter.GetProgress()
bone_segmenter.AddObserver(itk.ProgressEvent(), update_progress)

bone_segmenter.Update()

progress.bar_style = 'success'
progress.description = 'done.'

itkPointSetD3 not loaded from module ITKRegistrationCommon because of exception:
 module 'itk.ITKRegistrationCommonPython' has no attribute 'itkPointSetD3'
Template itk::LandmarkAtlasSegmentationFilter<itk::Image<signedshort,3>,itk::Image<signedshort,3>>
 already defined as <class 'itk.itkLandmarkAtlasSegmentationFilterPython.itkLandmarkAtlasSegmentationFilterISS3ISS3'>
 is redefined as <class 'itk.itkLandmarkAtlasSegmentationFilterPython.itkLandmarkAtlasSegmentationFilterISS3ISS3'>


In [37]:
label_image = bone_segmenter.GetOutput()
view(image=image,
     label_image=label_image,
     opacity_gaussians=opacity_gaussians,
     gradient_opacity=1.0,
     background=(0.6,0.6,0.6),
     camera=camera,
     ui_collapsed=True,
     )

Viewer(background=(0.6, 0.6, 0.6), camera=array([[-2.4597874 , 19.208582  , 45.884567  ],
       [ 4.855     ,…

In [32]:
viewer.camera

array([[-2.4597874 , 19.208582  , 45.884567  ],
       [ 4.855     ,  9.24      , 48.065998  ],
       [-0.22125234,  0.05076638,  0.97389436]], dtype=float32)